In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) SDK: AutoML Tabular Model Training Example

To use this Colaboratory notebook, you copy the notebook to your own Google Drive and open it with Colaboratory (or Colab). You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Colab automatically displays the return value of the last line in each cell. For more information about running notebooks in Colab, see the [Colab welcome page](https://colab.research.google.com/notebooks/welcome.ipynb).

This notebook demonstrate how to create an AutoML model based on a tabular dataset. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction,  storage or usage of other GCP products in connection with testing this SDK.

# Install AI Platform (Unified) SDK, Authenticate, and upload of a Dataset to your GCS bucket


After the SDK installation the kernel will be automatically restarted. You may see this error message `Your session crashed for an unknown reason` which is normal.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
MY_PROJECT = "YOUR PROJECT"
MY_STAGING_BUCKET = (
    "gs://YOUR BUCKET"  # bucket should be in same region as AI Platform (Unified)
)

The dataset we are using is the California Housing Dataset, available locally in Colab. For more information about this dataset please visit: https://developers.google.com/machine-learning/crash-course/california-housing-data-description

In [ ]:
!wget https://dl.google.com/mlcc/mledu-datasets/california_housing_train.csv

In [ ]:
TRAIN_FILE_NAME = "california_housing_train.csv"
!gsutil cp {TRAIN_FILE_NAME} {MY_STAGING_BUCKET}/data/

gcs_csv_path = f"{MY_STAGING_BUCKET}/data/{TRAIN_FILE_NAME}"

# Initialize AI Platform (Unified) SDK

Initialize the *client* for AI Platform (Unified)

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# Create a Managed Tabular Dataset from CSV

This section will create a dataset from a CSV file stored on your GCS bucket

In [ ]:
ds = aiplatform.TabularDataset.create(display_name="housing", gcs_source=[gcs_csv_path])

ds.resource_name

# Launch a Training Job to Create a Model

Once we have defined your training script, we will create a model.

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name="train-housing-automl_1",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=[
        {"numeric": {"column_name": "longitude"}},
        {"numeric": {"column_name": "latitude"}},
        {"numeric": {"column_name": "housing_median_age"}},
        {"numeric": {"column_name": "total_rooms"}},
        {"numeric": {"column_name": "total_bedrooms"}},
        {"numeric": {"column_name": "population"}},
        {"numeric": {"column_name": "households"}},
        {"numeric": {"column_name": "median_income"}},
    ],
    optimization_objective_recall_value=None,
    optimization_objective_precision_value=None,
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column="median_house_value",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name="house-value-prediction-model",
    disable_early_stopping=False,
    predefined_split_column_name=None,
)

# Deploy Your Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

# Predict on the Endpoint
- This sample is taken from an observation where median_house_value = 94600
- Note that the values are all strings. Since the original data was in CSV format, everything is treated as a string. The transformations passed into the Dataset inform AI Platform (Unified) to transform the inputs to numeric types.

In [ ]:
prediction = endpoint.predict(
    [
        {
            "longitude": "-124.35",
            "latitude": "40.54",
            "housing_median_age": "52.0",
            "total_rooms": "1820.0",
            "total_bedrooms": "300.0",
            "population": "806",
            "households": "270.0",
            "median_income": "3.014700",
        },
    ]
)
prediction